# Evaluation

In [8]:
import os
os.chdir("/Users/pavankumar/Projects/CNN-Classifier-End-to-end/")
%pwd

'/Users/pavankumar/Projects/CNN-Classifier-End-to-end'

## Entitiy

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int



## Configuration Manager

In [10]:
from CNN_CLASSIFIER.constants import *
from CNN_CLASSIFIER.utils import read_yaml, create_directories, save_json
import tensorflow as tf

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/PetImages",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
        )
        return eval_config


## Components

### Load the Model

In [11]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(

            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores ={"loss" : self.score[0],"accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)


In [15]:
try: 
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evalaution = Evaluation(val_config)
    evalaution.evaluate()
    evalaution.save_score()
except Exception as e:
    raise(e)

[2024-05-03 00:58:42,563: INFO: common: yaml file:config/config.yaml loaded Successfully]
[2024-05-03 00:58:42,565: INFO: common: yaml file:params.yaml loaded Successfully]
[2024-05-03 00:58:42,566: INFO: common: Created Directory at: artifacts]
Found 118 images belonging to 2 classes.
4/4 [==============================] - 9s 2s/step - loss: 21.6715 - accuracy: 0.5000
[2024-05-03 00:58:52,617: INFO: common: json file saved at: scores.json]
